# Imports

In [47]:
import numpy as np
import pandas as pd
import swifter
# import pandas_profiling
import pprint
import pickle

from datetime import datetime
import time

import pytz
from timezonefinder import TimezoneFinder

## Jupyter Configurations

In [48]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None)

%matplotlib inline

## Function Definitions

In [60]:
def is_dst(dt=None, timezone="UTC"):
    if dt is None:
        dt = datetime.utcnow()
    timezone = pytz.timezone(timezone)
    timezone_aware_date = timezone.localize(dt, is_dst=None)
    return timezone_aware_date.tzinfo._dst.seconds != 0

def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

## Read Data

In [50]:
%%time

df = pd.read_csv("US_Accidents_Dec19.csv")

print('Imported {:,} rows.'.format(len(df)))

Imported 2,974,335 rows.
CPU times: user 25.2 s, sys: 2.37 s, total: 27.5 s
Wall time: 27.9 s


# Data Cleaning

## Fill missing numeric values

In [51]:
%%time

df['Wind_Chill(F)'].fillna(df['Temperature(F)'], inplace=True)   #Went with Temp due to Wind_Chill.mean() giving values that didn't make sense.
df['Wind_Speed(mph)'].fillna(df['Wind_Speed(mph)'].mean(), inplace=True)
df['Precipitation(in)'].fillna(df['Precipitation(in)'].mean(), inplace=True)

CPU times: user 85 ms, sys: 14.5 ms, total: 99.5 ms
Wall time: 105 ms


## Prepare the Datetime values

In [52]:
%%time

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df['Weather_Timestamp'] = pd.to_datetime(df['Weather_Timestamp'])

df["Duration(s)"] = (df["End_Time"] - df["Start_Time"]) // pd.Timedelta('1s')
df['Start_Hour'] = df["Start_Time"].dt.hour
df['Start_Month'] = df["Start_Time"].dt.month
df['Start_Day'] = df["Start_Time"].dt.strftime("%A")

# Map Day name of week to number
d = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}

df['Start_Day'] = df['Start_Day'].map(d)

CPU times: user 19 s, sys: 750 ms, total: 19.8 s
Wall time: 19.8 s


## Fill missing timezones and futher refine

In [53]:
%%time

# Get Timezone info for missing. Also converts the timezones to more localized ones from the standard four.
# May need to revisit and convert back to the standard four US timezones.
df['Timezone'] = df['Timezone'].astype('str')
tf = TimezoneFinder()
df['Timezone'] = df.swifter.apply(lambda row: tf.timezone_at(lat=row['Start_Lat'], lng=row['Start_Lng']),axis=1)


CPU times: user 2min 59s, sys: 34.7 s, total: 3min 33s
Wall time: 3min 39s


## Add Daylight Saving Time (DST) feature

In [54]:
%%time

# Add Daylight Savings Time to allow for analysis.
df['DST'] = df.swifter.apply(lambda row: is_dst(row['Start_Time'], timezone=row['Timezone']), axis=1)


CPU times: user 18min 55s, sys: 1min 21s, total: 20min 16s
Wall time: 20min


## Prepare the categorical data

In [61]:
%%time

df['Side'] = df['Side'].astype(str)
df.at[df[~df['Side'].isin(['L','R'])].index, 'Side'] = 'U'  #Replace unknown values with U
# df['Side'] = df['Side'].astype('category')


# df['Airport_Code'] = df['Airport_Code'].astype('category')
# df['Wind_Direction'] = df['Wind_Direction'].astype('category')
# df['Weather_Condition'] = df['Weather_Condition'].astype('category')


# Standarize the Wind_Direction values
wordDict = {'South': 'S', 'West': 'W', 'North': 'N', 'East': 'E', 'Variable': 'VAR', 'Calm': 'CALM'}
df['Wind_Direction'].fillna('VAR', inplace=True)
df['Wind_Direction'] = df.swifter.apply(lambda row: multipleReplace(row['Wind_Direction'], wordDict), axis = 1)

CPU times: user 1.08 s, sys: 239 ms, total: 1.32 s
Wall time: 1.36 s


In [56]:
%%time

# Cannot convert these before fillna occurs

df['Severity'] = df['Severity'].astype(int)

df['Zipcode'] = df['Zipcode'].str.split('-').str[0].astype(str)  # Throw away the Zip-4

CPU times: user 4.17 s, sys: 572 ms, total: 4.74 s
Wall time: 5.11 s


In [57]:
%%time

for col in ['Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight']:
    df[col] = np.where(df[col].str.contains("Day"), True, False)

CPU times: user 6.85 s, sys: 948 ms, total: 7.8 s
Wall time: 8.03 s


In [58]:
%%time

# Drop unecessary columns

df.drop(['Source'], axis = 1, inplace = True) # Source doesn't seem to be useful as it's three values
df.drop(['End_Lat','End_Lng'], axis = 1, inplace = True) # Primarily NaN
df.drop(['Distance(mi)'], axis = 1, inplace = True) #Doesn't seem to be relevant as approx. 2M rows are 0.01

CPU times: user 1.96 s, sys: 1.7 s, total: 3.67 s
Wall time: 4.1 s


## Write out data to Pickle

In [63]:
%%time

outfile = open('US_Accidents_Dec19', 'wb')
pickle.dump(df, outfile)
outfile.close()

CPU times: user 6.01 s, sys: 3.82 s, total: 9.84 s
Wall time: 11 s
